# Item-based Models

In this file we are going to proceed with the building in the models. Machine learning models to build recommender systems. The models that are going to be built are Collaborative filtering using Item-based rating prediction (ItemKNN) and Item-based classification (ItemKNN).

Collaborative filtering is a technique used in recommendation systems to predict or classify items based on the preferences or behavior of similar users or items. Item-based Collaborative Filtering (CF) focuses on the similarity between items rather than users. There are two main approaches within item-based CF: rating prediction and classification.

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # to make jupyter print all outputs, not just the last one
from IPython.core.display import HTML # to pretty print pandas df and be able to copy them over (e.g. to ppt slides)

In [2]:
# We import the necessary libraries
import pandas as pd
import numpy as np
import os

In [3]:
# We print the directory where the file is located
print(os.getcwd())

c:\Users\Jaume\Documents\MDDB\SDM\SDfM---Jaume-and-Stijn 2\SDfM---Jaume-and-Stijn


In [4]:
# We set the directory to the cleaned folder
os.listdir(os.path.join('.', 'cleaned'))

['final_sample30_parquet',
 'final_sample50_parquet',
 'final_sample5_parquet',
 'movielens_parquet',
 'netflix_parquet',
 'sample_tenth_netflix',
 'unpacked_reviews_df_100k.parquet']

In [5]:
# We read the final_sample file and store it in a dataframe
df = pd.read_parquet('cleaned/sample_tenth_netflix')

In [6]:
# We print shape of the dataframe
df.shape

(143, 4)

In [7]:
# We print the first 5 rows of the dataframe
df.head()

,movieId,year,title,review_data
123,124,2000,Cold Blooded,"[{'date': 2002-04-01, 'rating': 3.0, 'userId':..."
1192,13434,1998,Beast Cops,"[{'date': 2003-02-20, 'rating': 2.0, 'userId':..."
462,4601,1994,The Scout,"[{'date': 2003-12-27, 'rating': 3.0, 'userId':..."
351,352,1997,Pick a Card,"[{'date': 2003-10-13, 'rating': 4.0, 'userId':..."
1057,9560,2000,New World Disorder,"[{'date': 2003-07-06, 'rating': 4.0, 'userId':..."


In [8]:
# We print the columns of the dataframe
df.columns

Index(['movieId', 'year', 'title', 'review_data'], dtype='object')

In [9]:
# We check if there are movies with less than 10 reviews, if there are we take them out to handle the cold start problem
df = df[df['review_data'].apply(lambda x: len(x) if x is not None else 0) > 10]

In [10]:
# Explode the review_data column
exploded_df = df.explode('review_data').reset_index(drop=True)

In [11]:
exploded_df.head()

exploded_df.shape

# We get the first row of exploded_df
exploded_df.iloc[0]

,movieId,year,title,review_data
0,124,2000,Cold Blooded,"{'date': 2002-04-01, 'rating': 3.0, 'userId': ..."
1,124,2000,Cold Blooded,"{'date': 2002-05-14, 'rating': 3.0, 'userId': ..."
2,124,2000,Cold Blooded,"{'date': 2001-06-06, 'rating': 3.0, 'userId': ..."
3,124,2000,Cold Blooded,"{'date': 2001-07-28, 'rating': 2.0, 'userId': ..."
4,124,2000,Cold Blooded,"{'date': 2002-02-04, 'rating': 1.0, 'userId': ..."


(799645, 4)

movieId                                                      124
year                                                        2000
title                                               Cold Blooded
review_data    {'date': 2002-04-01, 'rating': 3.0, 'userId': ...
Name: 0, dtype: object

In [12]:
# Convert the exploded dictionary column into separate columns
expanded_review_data = pd.json_normalize(exploded_df['review_data'])


In [13]:
expanded_review_data.head()

,date,rating,userId
0,2002-04-01,3.0,68845
1,2002-05-14,3.0,349975
2,2001-06-06,3.0,1619594
3,2001-07-28,2.0,2377588
4,2002-02-04,1.0,1361044


In [14]:
# Concatenate the expanded data with the original DataFrame
df = pd.concat([exploded_df.drop('review_data', axis=1), expanded_review_data], axis=1)

# Verify the shape of the final DataFrame
print("DataFrame shape:", df.shape)


DataFrame shape: (799645, 6)


In [15]:
df.head()

,movieId,year,title,date,rating,userId
0,124,2000,Cold Blooded,2002-04-01,3.0,68845
1,124,2000,Cold Blooded,2002-05-14,3.0,349975
2,124,2000,Cold Blooded,2001-06-06,3.0,1619594
3,124,2000,Cold Blooded,2001-07-28,2.0,2377588
4,124,2000,Cold Blooded,2002-02-04,1.0,1361044


In [16]:
# We print the number of unique users and movies
print(df['userId'].nunique())
print(df['movieId'].nunique())


254342
143


In [17]:
# Check if there are userIDs that haven't rated movies at all
df['userId'].value_counts().sort_values(ascending=False)

# Check if there are movieIDs that haven't been rated at all
df['movieId'].value_counts().sort_values(ascending=False)

userId
305344     142
387418     141
2439493    133
1664010    130
2118461    125
          ... 
1093522      1
401886       1
1753823      1
1707129      1
773753       1
Name: count, Length: 254342, dtype: int64

movieId
30       118413
4705      58536
290       47651
4727      41128
199       35509
          ...  
9249         83
9603         83
9372         71
13478        46
13405        29
Name: count, Length: 143, dtype: int64

In [18]:
# Check if there are NaN values in the rating column
df['rating'].isna().sum()

0

In [19]:
# Group the DataFrame by 'userId' and count the number of ratings per user
user_rating_counts = df.groupby('userId').size()

# Filter out users who have rated less than 5 movies
active_users = user_rating_counts[user_rating_counts >= 5].index.tolist()

# Select rows from the original DataFrame for active users
df = df[df['userId'].isin(active_users)]

# Display the filtered DataFrame
df.head()


,movieId,year,title,date,rating,userId
0,124,2000,Cold Blooded,2002-04-01,3.0,68845
4,124,2000,Cold Blooded,2002-02-04,1.0,1361044
6,124,2000,Cold Blooded,2005-08-06,3.0,1716677
10,124,2000,Cold Blooded,2001-12-20,2.0,1521983
12,124,2000,Cold Blooded,2001-12-07,2.0,650960


In [20]:
df.shape

(424670, 6)

In [21]:
# We take a sample of 10% of the data
df = df.sample(frac=0.05, random_state=42)

In [22]:
# Verify the shape of the final DataFrame
print("DataFrame shape:", df.shape)

DataFrame shape: (21234, 6)


In [23]:
# We move to the parquets folder to read the file with the Netflix genres

# We print the directory where the file is located
print(os.getcwd())

# We set the directory to the cleaned folder
os.listdir(os.path.join('.', 'parquets'))

# We read the netflix_genres file and store it in a dataframe
df_genres = pd.read_csv('parquets/netflix_genres.csv')

c:\Users\Jaume\Documents\MDDB\SDM\SDfM---Jaume-and-Stijn 2\SDfM---Jaume-and-Stijn


['movies_df', 'movie_title_df', 'netflix_df', 'netflix_genres.csv', 'users_df']

In [24]:
# We print the number of unique users and movies
print(df['userId'].nunique())
print(df['movieId'].nunique())

17037
139


In [25]:
# Merge the two DataFrames on the 'movieId' column, keeping all rows from the left DataFrame
df = pd.merge(df, df_genres, on='movieId', how='left')

df


,movieId,year,title,date,rating,userId,genres
0,290,2004,Harold and Kumar Go to White Castle,2005-05-22,1.0,239658,Adventure|Comedy
1,4506,1961,Breakfast at Tiffany's,2004-04-13,4.0,1981621,Comedy|Drama|Romance
2,4727,1979,Apocalypse Now Redux,2004-05-25,5.0,2629246,Drama|Mystery|War
3,13629,1951,Alice in Wonderland,2004-08-13,3.0,857778,Animation|Adventure|Family|Fantasy|Musical
4,4727,1979,Apocalypse Now Redux,2003-11-28,5.0,157357,Drama|Mystery|War
...,...,...,...,...,...,...,...
21229,4506,1961,Breakfast at Tiffany's,2005-11-08,3.0,327400,Comedy|Drama|Romance
21230,199,1978,The Deer Hunter,2004-07-05,3.0,1602132,Drama|War
21231,4805,1993,True Romance,2004-01-04,3.0,1763574,Crime|Drama|Romance|Thriller
21232,4761,1981,The Cannonball Run,2005-05-06,3.0,1037245,Action|Comedy|Sport


In [26]:
# We print the number of unique users and movies
print(df['userId'].nunique())
print(df['movieId'].nunique())

17037
139


In [27]:
# We split the content of the genres by the '|'
df['genres'] = df['genres'].str.split('|')

df['genres']

0                                     [Adventure, Comedy]
1                                [Comedy, Drama, Romance]
2                                   [Drama, Mystery, War]
3        [Animation, Adventure, Family, Fantasy, Musical]
4                                   [Drama, Mystery, War]
                               ...                       
21229                            [Comedy, Drama, Romance]
21230                                        [Drama, War]
21231                   [Crime, Drama, Romance, Thriller]
21232                             [Action, Comedy, Sport]
21233                          [Biography, Comedy, Drama]
Name: genres, Length: 21234, dtype: object

In [28]:
# One-hot encode the 'genres' column
df = df.join(df['genres'].str.join('|').str.get_dummies())

# Drop the original 'genres' column
df.drop('genres', axis=1, inplace=True)

print(df)


       movieId  year                                title        date  rating  \
0          290  2004  Harold and Kumar Go to White Castle  2005-05-22     1.0   
1         4506  1961               Breakfast at Tiffany's  2004-04-13     4.0   
2         4727  1979                 Apocalypse Now Redux  2004-05-25     5.0   
3        13629  1951                  Alice in Wonderland  2004-08-13     3.0   
4         4727  1979                 Apocalypse Now Redux  2003-11-28     5.0   
...        ...   ...                                  ...         ...     ...   
21229     4506  1961               Breakfast at Tiffany's  2005-11-08     3.0   
21230      199  1978                      The Deer Hunter  2004-07-05     3.0   
21231     4805  1993                         True Romance  2004-01-04     3.0   
21232     4761  1981                   The Cannonball Run  2005-05-06     3.0   
21233      311  1994                              Ed Wood  2005-09-12     4.0   

        userId  Action  Adv

In [29]:
df.head

<bound method NDFrame.head of        movieId  year                                title        date  rating  \
0          290  2004  Harold and Kumar Go to White Castle  2005-05-22     1.0   
1         4506  1961               Breakfast at Tiffany's  2004-04-13     4.0   
2         4727  1979                 Apocalypse Now Redux  2004-05-25     5.0   
3        13629  1951                  Alice in Wonderland  2004-08-13     3.0   
4         4727  1979                 Apocalypse Now Redux  2003-11-28     5.0   
...        ...   ...                                  ...         ...     ...   
21229     4506  1961               Breakfast at Tiffany's  2005-11-08     3.0   
21230      199  1978                      The Deer Hunter  2004-07-05     3.0   
21231     4805  1993                         True Romance  2004-01-04     3.0   
21232     4761  1981                   The Cannonball Run  2005-05-06     3.0   
21233      311  1994                              Ed Wood  2005-09-12     4.0  

In [30]:
# We have to rename all the genre columns and make them start with 'genre_'
df.columns

genre_columns = ['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 
        'Drama', 'Family', 'Fantasy', 'Film-Noir', 'History', 'Horror', 'Music',
       'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Short', 'Sport', 'Thriller',
       'War']

# Define the new column names with 'genre_' prefix
new_columns = {'{}'.format(col): 'genre_{}'.format(col) for col in genre_columns}

# Rename the columns in the DataFrame
df.rename(columns=new_columns, inplace=True)

# Now, df will have the genre columns starting with 'genre_'
print(df.columns)


Index(['movieId', 'year', 'title', 'date', 'rating', 'userId', 'Action',
       'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary',
       'Drama', 'Family', 'Fantasy', 'Film-Noir', 'History', 'Horror', 'Music',
       'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Short', 'Sport', 'Thriller',
       'War'],
      dtype='object')

Index(['movieId', 'year', 'title', 'date', 'rating', 'userId', 'genre_Action',
       'genre_Adventure', 'genre_Animation', 'genre_Biography', 'genre_Comedy',
       'genre_Crime', 'genre_Documentary', 'genre_Drama', 'genre_Family',
       'genre_Fantasy', 'genre_Film-Noir', 'genre_History', 'genre_Horror',
       'genre_Music', 'genre_Musical', 'genre_Mystery', 'genre_Romance',
       'genre_Sci-Fi', 'genre_Short', 'genre_Sport', 'genre_Thriller',
       'genre_War'],
      dtype='object')


In [31]:
# We print the number of unique users and movies
print(df['userId'].nunique())
print(df['movieId'].nunique())

17037
139


# Item-based Rating Prediction (ItemKNN)

### Step 1: User-Item matrix construction
The first thing we have to do is build the user-item matrix:

- Choose a similarity metric to calculate the similarity between items. Common metrics include cosine similarity, Pearson correlation coefficient, and Jaccard similarity.
- Calculate the similarity between each pair of items based on the ratings provided by users. This will result in an item-item similarity matrix.

#### Collaborative Filtering with Cosine Similarity

This code snippet demonstrates how to perform item-based collaborative filtering using cosine similarity. Collaborative filtering is a technique commonly used in recommendation systems to predict a user's preferences for items based on the preferences of similar users/items.

##### Libraries Used
- **pandas**: A powerful data manipulation library in Python.
- **numpy**: A library for numerical computing in Python.
- **sklearn.metrics.pairwise.cosine_similarity**: A function from scikit-learn used to compute the cosine similarity between vectors.

In [32]:
import pandas as pd
import numpy as np

# Convert userId column to integer type if needed
if df['userId'].dtype != int:
    df['userId'] = df['userId'].astype(int)

# Get all unique user IDs and movie IDs
all_user_ids = np.unique(df['userId'])
all_movie_ids = np.unique(df['movieId'])

# Determine the number of unique users and movies
num_users = len(all_user_ids)
num_movies = len(all_movie_ids)

# Create a dictionary to store ratings for each user
user_ratings = {}

# Iterate through the DataFrame and populate the user_ratings dictionary
for _, row in df.iterrows():
    user_id = row['userId']
    movie_id = row['movieId']
    rating = row['rating']
    
    # Initialize the user's ratings list if not already present
    if user_id not in user_ratings:
        user_ratings[user_id] = np.zeros(num_movies)
    
    # Assign the rating to the corresponding movie index
    user_ratings[user_id][np.where(all_movie_ids == movie_id)[0][0]] = rating

# Create user-item matrix from the dictionary
item_user_matrix = pd.DataFrame(user_ratings.values(), index=user_ratings.keys(), columns=all_movie_ids)

# Convert user-item matrix to NumPy array for faster computation
item_user_array = item_user_matrix.to_numpy()


In [33]:
item_user_matrix.head()

,30,33,44,45,50,52,66,68,71,77,...,13576,13581,13594,13596,13625,13626,13629,13656,13668,13673
239658,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1981621,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2629246,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
857778,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
157357,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
item_user_array

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Train-test split

The train-val-test split is a technique used in machine learning to evaluate the performance of a model. It involves dividing the dataset into three subsets: the training set, the validation set, and the test set.

The training set is used to train the model and optimize its parameters.
The validation set is used to fine-tune the model and select the best hyperparameters.
The test set is used to evaluate the final performance of the model on unseen data.
By using a train-val-test split, we can assess the model's performance on unseen data and ensure that it generalizes well to new examples. It helps prevent overfitting and provides a more reliable estimate of the model's performance in real-world scenarios.

In [35]:
from sklearn.model_selection import train_test_split
# Split the data into training and test sets
train_val, test = train_test_split(item_user_array, test_size=0.2, random_state=42)

# Split the training set into training and validation sets
train, val = train_test_split(train_val, test_size=0.2, random_state=42)

# Print the shapes of the datasets
print("Training set shape:", train.shape)
print("Validation set shape:", val.shape)
print("Test set shape:", test.shape)

# We are also going to do the split for the matrix df
# Split the user-item matrix into training and test sets
train_val_matrix, test_matrix = train_test_split(item_user_matrix, test_size=0.2, random_state=42)

# Split the training set matrix into training and validation sets
train_matrix, val_matrix = train_test_split(train_val_matrix, test_size=0.2, random_state=42)

# We print a ' ' to give some space inbetween lines
print(' ')

# Print the shapes of the matrix datasets
print("Training set matrix shape:", train_matrix.shape)
print("Validation set matrix shape:", val_matrix.shape)
print("Test set matrix shape:", test_matrix.shape)


Training set shape: (10903, 139)
Validation set shape: (2726, 139)
Test set shape: (3408, 139)
 
Training set matrix shape: (10903, 139)
Validation set matrix shape: (2726, 139)
Test set matrix shape: (3408, 139)


In [36]:
train_matrix

,30,33,44,45,50,52,66,68,71,77,...,13576,13581,13594,13596,13625,13626,13629,13656,13668,13673
2227565,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1950639,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1093688,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2285835,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2085211,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2627144,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1599231,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1849553,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1298946,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
train

array([[0., 0., 0., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [38]:
# Calculate cosine similarity between items using NumPy functions
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    similarity = dot_product / (norm_a * norm_b)
    return similarity

# Calculate item-item similarity matrix for train data
#item_similarity_matrix_train = np.zeros((train.shape[1], train.shape[1]))

#for i in range(train.shape[1]):
 #   for j in range(i, train.shape[1]):
  #      item_similarity_matrix_train[i, j] = cosine_similarity(train[:, i], train[:, j])
   #     item_similarity_matrix_train[j, i] = item_similarity_matrix_train[i, j]

# Adjusting the mapping to start indexing from 0
user_id_to_index = {user_id: i for i, user_id in enumerate(item_user_matrix.index)}
index_to_user_id = {i: user_id for i, user_id in enumerate(item_user_matrix.index)}

# Create a mapping from movie IDs to indices
movie_id_to_index = {movie_id: i for i, movie_id in enumerate(item_user_matrix.columns)}
index_to_movie_id = {i: movie_id for i, movie_id in enumerate(item_user_matrix.columns)}




### Similarity Calculation utilizing Genres:



In [39]:
import pandas as pd

# Function to create item-genre matrix
def create_item_genre_matrix(df, train):
    # Extract genre columns from the dataframe
    genre_columns = [col for col in df.columns if col.startswith('genre_')]
    
    # Initialize item-genre matrix with zeros
    item_genre_matrix = pd.DataFrame(0, index=genre_columns, columns=train.columns)
    
    # Fill the matrix with genre information
    for i, row in df.iterrows():
        
        item_id = row['movieId']  # 'movieId' is the column containing item IDs
        genres = row[genre_columns].values
        item_genre_matrix[item_id] = genres
    
    return item_genre_matrix


# df is the original dataframe with genre information and train_matrix is our train set dataframe
item_genre_matrix = create_item_genre_matrix(df, train_matrix)


In [40]:
item_genre_matrix

item_genre_matrix.shape

,30,33,44,45,50,52,66,68,71,77,...,13576,13581,13594,13596,13625,13626,13629,13656,13668,13673
genre_Action,0,0,0,0,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
genre_Adventure,0,0,0,0,1,0,0,1,0,1,...,0,0,1,0,0,0,1,0,0,1
genre_Animation,0,1,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,1,0,0,1
genre_Biography,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
genre_Comedy,1,1,0,1,0,0,0,1,0,0,...,0,1,0,0,1,0,0,0,0,1
genre_Crime,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
genre_Documentary,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
genre_Drama,1,0,1,1,1,0,0,1,0,0,...,1,0,1,1,1,1,0,0,0,0
genre_Family,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,1,0,0,1
genre_Fantasy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1


(22, 139)

#### Collaborative Filtering with Cosine Similarity

This code snippet demonstrates how to perform item-based collaborative filtering using cosine similarity. Collaborative filtering is a technique commonly used in recommendation systems to predict a user's preferences for items based on the preferences of similar users/items.


In [41]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def calculate_item_similarity_matrix(data):
    # Calculate cosine similarity between items
    item_similarity_matrix = cosine_similarity(data.T)
    np.fill_diagonal(item_similarity_matrix, 1)  # Set diagonal values to 1
    return item_similarity_matrix

# 'train' is the train set data
item_similarity_matrix_train = calculate_item_similarity_matrix(train)
print(item_similarity_matrix_train)


[[1.         0.00521925 0.00289134 ... 0.         0.00158359 0.01632566]
 [0.00521925 1.         0.         ... 0.         0.         0.00650294]
 [0.00289134 0.         1.         ... 0.         0.         0.01306498]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.00158359 0.         0.         ... 0.         1.         0.        ]
 [0.01632566 0.00650294 0.01306498 ... 0.         0.         1.        ]]


In [42]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def calculate_item_similarity_genres(item_genre_matrix):
    # Calculate cosine similarity between items
    item_similarity_matrix = cosine_similarity(item_genre_matrix.T)
    np.fill_diagonal(item_similarity_matrix, 1)  # Set diagonal values to 1
    return item_similarity_matrix

# item_genre_matrix is the item-genre matrix
item_similarity_genres_matrix = calculate_item_similarity_genres(item_genre_matrix)


In [43]:
item_similarity_matrix_train

array([[1.        , 0.00521925, 0.00289134, ..., 0.        , 0.00158359,
        0.01632566],
       [0.00521925, 1.        , 0.        , ..., 0.        , 0.        ,
        0.00650294],
       [0.00289134, 0.        , 1.        , ..., 0.        , 0.        ,
        0.01306498],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.00158359, 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.01632566, 0.00650294, 0.01306498, ..., 0.        , 0.        ,
        1.        ]])

In [44]:
item_similarity_genres_matrix

array([[1.        , 0.40824829, 0.57735027, ..., 0.        , 0.        ,
        0.25819889],
       [0.40824829, 1.        , 0.        , ..., 0.        , 0.        ,
        0.63245553],
       [0.57735027, 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.25819889, 0.63245553, 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [45]:
movie_id_to_index

{30: 0,
 33: 1,
 44: 2,
 45: 3,
 50: 4,
 52: 5,
 66: 6,
 68: 7,
 71: 8,
 77: 9,
 79: 10,
 102: 11,
 116: 12,
 124: 13,
 164: 14,
 169: 15,
 185: 16,
 199: 17,
 209: 18,
 219: 19,
 221: 20,
 232: 21,
 234: 22,
 238: 23,
 240: 24,
 248: 25,
 271: 26,
 290: 27,
 298: 28,
 310: 29,
 311: 30,
 333: 31,
 340: 32,
 345: 33,
 352: 34,
 354: 35,
 355: 36,
 359: 37,
 4506: 38,
 4509: 39,
 4513: 40,
 4521: 41,
 4554: 42,
 4571: 43,
 4578: 44,
 4592: 45,
 4600: 46,
 4601: 47,
 4610: 48,
 4625: 49,
 4636: 50,
 4666: 51,
 4667: 52,
 4671: 53,
 4693: 54,
 4705: 55,
 4727: 56,
 4729: 57,
 4730: 58,
 4733: 59,
 4738: 60,
 4744: 61,
 4746: 62,
 4752: 63,
 4758: 64,
 4761: 65,
 4768: 66,
 4777: 67,
 4788: 68,
 4790: 69,
 4805: 70,
 9214: 71,
 9218: 72,
 9233: 73,
 9240: 74,
 9249: 75,
 9255: 76,
 9270: 77,
 9305: 78,
 9311: 79,
 9317: 80,
 9319: 81,
 9329: 82,
 9331: 83,
 9339: 84,
 9348: 85,
 9367: 86,
 9368: 87,
 9372: 88,
 9385: 89,
 9389: 90,
 9390: 91,
 9403: 92,
 9412: 93,
 9436: 94,
 9441: 95,
 94

In [46]:
print(index_to_user_id)

{0: 239658, 1: 1981621, 2: 2629246, 3: 857778, 4: 157357, 5: 1147908, 6: 2565975, 7: 1918987, 8: 2511879, 9: 1298511, 10: 896306, 11: 2210638, 12: 372584, 13: 1142398, 14: 1515210, 15: 1665250, 16: 676278, 17: 941528, 18: 577616, 19: 568045, 20: 1591884, 21: 2197296, 22: 1276647, 23: 1467176, 24: 2267391, 25: 504620, 26: 875771, 27: 261650, 28: 2345122, 29: 1315379, 30: 1728300, 31: 130843, 32: 835915, 33: 297901, 34: 1118220, 35: 1739636, 36: 638866, 37: 1473980, 38: 2371817, 39: 49355, 40: 500249, 41: 1724291, 42: 806150, 43: 1326360, 44: 1756796, 45: 1923588, 46: 1532336, 47: 1262971, 48: 2629689, 49: 234959, 50: 330460, 51: 1984949, 52: 1005911, 53: 1178353, 54: 1873315, 55: 459816, 56: 2014999, 57: 1392168, 58: 1621546, 59: 1525560, 60: 1868345, 61: 2390444, 62: 1194594, 63: 1091730, 64: 2008504, 65: 1868327, 66: 844391, 67: 714459, 68: 1771252, 69: 936928, 70: 2012897, 71: 2187053, 72: 353477, 73: 2461201, 74: 2425195, 75: 2413192, 76: 1875225, 77: 1017698, 78: 123384, 79: 385778

In [47]:
print(user_id_to_index)

{239658: 0, 1981621: 1, 2629246: 2, 857778: 3, 157357: 4, 1147908: 5, 2565975: 6, 1918987: 7, 2511879: 8, 1298511: 9, 896306: 10, 2210638: 11, 372584: 12, 1142398: 13, 1515210: 14, 1665250: 15, 676278: 16, 941528: 17, 577616: 18, 568045: 19, 1591884: 20, 2197296: 21, 1276647: 22, 1467176: 23, 2267391: 24, 504620: 25, 875771: 26, 261650: 27, 2345122: 28, 1315379: 29, 1728300: 30, 130843: 31, 835915: 32, 297901: 33, 1118220: 34, 1739636: 35, 638866: 36, 1473980: 37, 2371817: 38, 49355: 39, 500249: 40, 1724291: 41, 806150: 42, 1326360: 43, 1756796: 44, 1923588: 45, 1532336: 46, 1262971: 47, 2629689: 48, 234959: 49, 330460: 50, 1984949: 51, 1005911: 52, 1178353: 53, 1873315: 54, 459816: 55, 2014999: 56, 1392168: 57, 1621546: 58, 1525560: 59, 1868345: 60, 2390444: 61, 1194594: 62, 1091730: 63, 2008504: 64, 1868327: 65, 844391: 66, 714459: 67, 1771252: 68, 936928: 69, 2012897: 70, 2187053: 71, 353477: 72, 2461201: 73, 2425195: 74, 2413192: 75, 1875225: 76, 1017698: 77, 123384: 78, 385778: 79

In [48]:
item_user_array.shape

(17037, 139)

In [49]:
item_user_matrix.shape

(17037, 139)

In [50]:
# We print the item_similarity_matrix
item_similarity_matrix_train

array([[1.        , 0.00521925, 0.00289134, ..., 0.        , 0.00158359,
        0.01632566],
       [0.00521925, 1.        , 0.        , ..., 0.        , 0.        ,
        0.00650294],
       [0.00289134, 0.        , 1.        , ..., 0.        , 0.        ,
        0.01306498],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.00158359, 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.01632566, 0.00650294, 0.01306498, ..., 0.        , 0.        ,
        1.        ]])

In [51]:
item_similarity_matrix_train.shape

(139, 139)

### Matrices Summation:

Once we have both similarity matrices (the one from the ratings and the one from the genres) we can add them with a weighted sumation.

In [52]:
def combine_similarity_matrices(similarity_matrix1, similarity_matrix2, weight1 = 0.5, weight2 = 0.5):
    # Check if the matrices have the same shape
    if similarity_matrix1.shape != similarity_matrix2.shape:
        raise ValueError("Matrices must have the same shape.")

    # Combine matrices using weighted summation
    combined_similarity_matrix = (weight1 * similarity_matrix1) + (weight2 * similarity_matrix2)
    return combined_similarity_matrix

# Define weights for each matrix
weight_ratings = 0.5
weight_genres = 0.5

# Combine the similarity matrices
item_similarity_matrix_train = combine_similarity_matrices(item_similarity_matrix_train, item_similarity_genres_matrix, weight_ratings, weight_genres)

# Print or use the combined similarity matrix as needed
print("Combined Similarity Matrix:\n", item_similarity_matrix_train)


Combined Similarity Matrix:
 [[1.00000000e+00 2.06733770e-01 2.90120806e-01 ... 0.00000000e+00
  7.91795668e-04 1.37262276e-01]
 [2.06733770e-01 1.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 3.19479236e-01]
 [2.90120806e-01 0.00000000e+00 1.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 6.53249225e-03]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [7.91795668e-04 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  1.00000000e+00 0.00000000e+00]
 [1.37262276e-01 3.19479236e-01 6.53249225e-03 ... 0.00000000e+00
  0.00000000e+00 1.00000000e+00]]


### Step 2: Neighborhood Selection
- Determine the neighborhood size, i.e., the number of most similar items to consider when predicting ratings for a target item.
- Select the most similar items for each item in our dataset based on their calculated similarities. This forms the neighborhood for each item.

#### Item-Based Neighborhoods and Ratings Aggregation

This code following snippet enhances the previous item-based collaborative filtering approach by considering ratings aggregation within the item neighborhoods.

##### Steps:

1. **Defining Neighborhood Size**:
   - The variable `neighborhood_size` determines the number of most similar items to consider in the neighborhood.

2. **Initializing Data Structure**:
   - An empty dictionary `item_neighborhoods` is initialized to store the neighborhoods for each item.

3. **Iterating Over Items**:
   - For each movie in the dataset:
     - All ratings for the current movie are extracted from the DataFrame (`df`).
     - Ratings aggregation is performed. In this example, the average rating for the movie is computed, but other aggregation methods can be used.
     - The similarity scores for the current movie are retrieved from the precomputed `item_similarity_matrix`.
     - Similarity scores are sorted in descending order, and the indices of the most similar items (excluding itself) are obtained.
     - These indices are converted back to movie IDs, forming the neighborhood for the current item.
     - The neighborhood for the current item is stored in the `item_neighborhoods` dictionary.

4. **Output**:
   - `item_neighborhoods`: A dictionary where keys are movie IDs, and values are lists of movie IDs representing the neighborhood of each item. Each movie's neighborhood includes movies with similar ratings and content.

##### Note:
- This approach considers both similarity in ratings and content (as captured by cosine similarity) when building item neighborhoods.
- Aggregating ratings within item neighborhoods helps in providing more personalized recommendations.
- The choice of rating aggregation method (e.g., mean, median) can impact the quality of recommendations and may need to be adjusted based on the characteristics of the dataset and user preferences.


In [53]:
import numpy as np

def neighborhood_selection(item_user_array, index_to_movie_id, item_similarity_matrix, neighborhood_size=5):
    # Initialize an empty dictionary to store item neighborhoods
    item_neighborhoods = {}

    # Iterate over each item (movie) index in the dataset
    for movie_index in range(item_user_array.shape[1]):
        # Convert the item index to movie ID
        movie_id = index_to_movie_id[movie_index]

        # Extract all ratings for the current movie
        movie_ratings = item_user_array[:, movie_index]

        # Aggregate ratings (e.g., compute the mean rating)
        movie_avg_rating = np.mean(movie_ratings)

        # Retrieve similarity scores for the current movie
        similarity_scores = item_similarity_matrix[movie_index]

        # Sort similarity scores in descending order and get indices of most similar items
        most_similar_indices = np.argsort(similarity_scores)[::-1][1:neighborhood_size+1]

        # Convert indices back to movie IDs to form the neighborhood
        neighborhood = [index_to_movie_id[idx] for idx in most_similar_indices]

        # Store the neighborhood for the current item in the item_neighborhoods dictionary
        item_neighborhoods[movie_id] = neighborhood

    return item_neighborhoods

# Example usage:
item_neighborhoods_train = neighborhood_selection(item_user_array, index_to_movie_id, item_similarity_matrix_train)
print(item_neighborhoods_train)


{30: [4506, 13625, 4790, 45, 352], 33: [4625, 9319, 13581, 13673, 68], 44: [232, 9317, 102, 9331, 4746], 45: [4506, 13625, 30, 4790, 352], 50: [9311, 352, 13626, 164, 199], 52: [9552, 9305, 4733, 9249, 199], 66: [9319, 333, 13543, 13468, 13576], 68: [9348, 13673, 209, 13478, 77], 71: [4610, 9403, 9249, 9233, 9218], 77: [209, 9348, 13478, 68, 13458], 79: [13596, 13409, 124, 4805, 4521], 102: [4729, 44, 219, 232, 9331], 116: [4790, 4601, 45, 311, 352], 124: [13409, 13596, 4578, 4521, 4805], 164: [13383, 13626, 9550, 30, 45], 169: [4667, 185, 333, 4777, 4752], 185: [169, 4667, 13497, 271, 9233], 199: [4727, 9311, 44, 9317, 9331], 209: [77, 9348, 13478, 68, 4730], 219: [4746, 44, 4729, 9317, 102], 221: [4738, 13478, 9603, 116, 13489], 232: [9317, 44, 9331, 219, 102], 234: [13673, 4571, 4610, 4601, 4600], 238: [4744, 9249, 4554, 4733, 13497], 240: [4601, 4506, 30, 4790, 13625], 248: [4671, 9331, 52, 13625, 45], 271: [340, 9603, 124, 185, 4752], 290: [13478, 4625, 9319, 13673, 13497], 298: [

### Step 4: Rating Prediction

- For each target item and user pair where the user hasn't rated the target item:
Identify the neighborhood of similar items to the target item.
- Predict the rating for the target item using a weighted average of the ratings of the items in its neighborhood, where the weights are the similarities between the items and the target item.
- Adjust the prediction based on the user's average rating or other normalization techniques, if necessary.

In [54]:
# Train our model and create the item_neighborhoods dictionary using the training data (steps 1-3)

# Initialize an empty DataFrame to store predicted ratings
predicted_ratings_train = pd.DataFrame(index=train_matrix.index, columns=train_matrix.columns)

# Iterate over each user-item pair in the training set
for user_id, user_ratings in train_matrix.iterrows():
    for movie_id, rating in user_ratings.items():
        
        # Check if the movie has a neighborhood defined
        if movie_id in item_neighborhoods_train:
            neighborhood = item_neighborhoods_train[movie_id]
            
            # Filter out movies from the neighborhood that the user has rated in the training set
            filtered_neighborhood = [neighbor_movie_id for neighbor_movie_id in neighborhood if neighbor_movie_id in user_ratings.index and user_ratings.loc[neighbor_movie_id] != 0]
            
            # Check if there are valid indices in the filtered neighborhood
            if len(filtered_neighborhood) > 0:
                # Calculate the predicted rating for the target movie based on the neighborhood
                neighbor_ratings = [user_ratings.loc[neighbor_movie_id] for neighbor_movie_id in filtered_neighborhood]
                
                # Calculate the mean rating using only the movies in the neighborhood that have been rated by the user
                predicted_rating = np.mean(neighbor_ratings)
            else:
                # If the filtered neighborhood is empty, assign the mean rating of all movies rated by the user
                predicted_rating = user_ratings[user_ratings != 0].mean()
            
            # Assign the predicted rating to the corresponding cell in the DataFrame
            predicted_ratings_train.at[user_id, movie_id] = predicted_rating

# Fill NaN values with mean ratings across all users
predicted_ratings_train.fillna(predicted_ratings_train.mean().mean(), inplace=True)

# Display the head of the predicted_ratings DataFrame
predicted_ratings_train.head()


C:\Users\Jaume\AppData\Local\Temp\ipykernel_12960\1768794802.py:36: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  predicted_ratings_train.fillna(predicted_ratings_train.mean().mean(), inplace=True)


,30,33,44,45,50,52,66,68,71,77,...,13576,13581,13594,13596,13625,13626,13629,13656,13668,13673
2227565,4.000000,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
1950639,3.557844,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
1093688,3.000000,3.5,3.5,3.0,3.5,3.5,3.5,3.5,3.5,3.5,...,4.0,3.5,4.0,4.0,3.0,3.5,3.5,3.5,3.5,3.5
2285835,3.000000,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2085211,3.500000,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,...,4.0,3.5,4.0,4.0,3.5,3.5,3.5,3.5,3.5,3.5


In [55]:
# We count the NaN values in the predicted_ratings_train dataframe
predicted_ratings_train.isnull().sum().sum()

0

In [56]:
def get_top_recommendations(user_id, predicted_ratings, df):
    """
    Get top movie recommendations for a given user using predicted ratings.

    Parameters:
        user_id (int): ID of the user for whom recommendations are to be generated.
        predicted_ratings (pd.DataFrame): DataFrame containing predicted ratings for users and movies.
        df (pd.DataFrame): DataFrame containing movie ratings.

    Returns:
        list: Top movie titles recommended for the user.
    """
    # Ensure that predicted_ratings is a DataFrame
    if not isinstance(predicted_ratings, pd.DataFrame):
        raise ValueError("predicted_ratings must be a pandas DataFrame")

    # Check if the user ID exists in the predicted ratings DataFrame's index
    if user_id not in predicted_ratings.index:
        # If the user ID doesn't exist, return an empty list
        return ["The user doesn't exist"]

    # Get the predicted ratings for the user
    user_predicted_ratings = predicted_ratings.loc[user_id]
    
    # Filter out the movies that the user has already seen
    seen_movies = set(df[df['userId'] == user_id]['movieId'])
    unseen_movies = [movie_id for movie_id in user_predicted_ratings.index if movie_id not in seen_movies]
    
    # Check if there are unseen movies with predicted ratings
    if not unseen_movies:
        # If all movies have been seen, return an empty list
        return []
    
    # Sort the unseen movies by predicted rating in descending order
    sorted_unseen_movies = user_predicted_ratings[unseen_movies].sort_values(ascending=False)
    
    # Get the top 5 movie titles
    top_movie_ids = sorted_unseen_movies.head(5).index.tolist()
    
    # Get the unique movie titles corresponding to the top movie IDs
    top_movie_titles = set(df[df['movieId'].isin(top_movie_ids)]['title'])
    
    return list(top_movie_titles)[:5]  # Return only the first 5 unique movie titles


In [57]:
top_recommendations = []

# Iterate over each row (user) in the train_matrix
for user_id in train_matrix.index:
    # Get the actual user ID corresponding to the user index
    recommendations = get_top_recommendations(user_id, predicted_ratings_train, df)
    top_recommendations.append(recommendations)

# Output: List of top movie recommendations for each user in the train matrix
top_recommendations

# print size of top_recommendations
len(top_recommendations)


[["Something's Gotta Give",
  'Striptease',
  'Final Justice',
  'Legend of Crystania: The Chaos Ring',
  'Full Metal Panic FUMOFFU'],
 ['Benedict Arnold: A Question of Honor',
  'Striptease',
  'Aqua Teen Hunger Force: Vol. 1',
  'Kazaam',
  'Legend of Crystania: The Chaos Ring'],
 ['Red Lights',
  'Dancer in the Dark',
  'The Killing',
  'Striptease',
  'Sadomania'],
 ["Something's Gotta Give",
  'Striptease',
  'Final Justice',
  'Legend of Crystania: The Chaos Ring',
  'Full Metal Panic FUMOFFU'],
 ['Monument Ave.',
  'Wire in the Blood: Justice Painted Blind',
  'Dancer in the Dark',
  "I'll Sleep When I'm Dead",
  'The Killing'],
 ['Benedict Arnold: A Question of Honor',
  'Striptease',
  'Aqua Teen Hunger Force: Vol. 1',
  'Kazaam',
  'Legend of Crystania: The Chaos Ring'],
 ['Videodrome',
  'Like Father, Like Son',
  'Congo',
  'The Texas Chainsaw Massacre',
  'Full Metal Panic FUMOFFU'],
 ["Something's Gotta Give",
  'Benedict Arnold: A Question of Honor',
  'Striptease',
  'K

10903

### Step 5: Model Evaluation
- We evaluate the performance of our ItemKNN algorithm using appropriate evaluation metrics such as Root Mean Squared Error (RMSE), Mean Absolute Error (MAE), or others.
- Split our dataset into training and testing sets to assess the model's predictive accuracy on unseen data.

In [58]:
train.shape

predicted_ratings_train.shape

(10903, 139)

(10903, 139)

In [59]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

def evaluate_model(train_matrix, predicted_ratings):
    """
    Evaluate the model's performance on the training data.

    Parameters:
        train_matrix (numpy.ndarray): Item-user matrix from the training data.
        predicted_ratings (pandas.DataFrame or numpy.ndarray): Predicted ratings DataFrame or array for the training data.

    Returns:
        float: Mean Absolute Error (MAE) on the training data.
        float: Root Mean Squared Error (RMSE) on the training data.
    """
    # Convert predicted_ratings to a numpy array if it's a DataFrame
    if isinstance(predicted_ratings, pd.DataFrame):
        predicted_ratings = predicted_ratings.to_numpy()

    # Ensure train_matrix and predicted_ratings have the same shape
    assert train_matrix.shape == predicted_ratings.shape, "Shapes of train_matrix and predicted_ratings are not consistent."

    # Initialize lists to store true and predicted ratings
    true_ratings = []
    pred_ratings = []

    # Iterate over each user and their ratings
    for user_id, user_ratings in enumerate(train_matrix):
        for movie_id, rating in enumerate(user_ratings):
            # Skip unrated movies
            if rating == 0:
                continue

            # Check if the indices are within the bounds of the predicted ratings array
            if user_id < predicted_ratings.shape[0] and movie_id < predicted_ratings.shape[1]:
                # Get the predicted rating for the corresponding movie
                pred_rating = predicted_ratings[user_id, movie_id]

                # Append the true and predicted ratings
                true_ratings.append(rating)
                pred_ratings.append(pred_rating)

    # Convert lists to numpy arrays
    true_ratings = np.array(true_ratings)
    pred_ratings = np.array(pred_ratings)

    # Calculate evaluation metrics
    mae = mean_absolute_error(true_ratings, pred_ratings)
    rmse = np.sqrt(mean_squared_error(true_ratings, pred_ratings))

    return mae, rmse

# train is the numpy array representing the item-user matrix
# predicted_ratings_train is the DataFrame or numpy array representing the predicted ratings for the training data

# Evaluate the model
mae, rmse = evaluate_model(train, predicted_ratings_train)

print("Mean Absolute Error (MAE) on Training Data:", mae)
print("Root Mean Squared Error (RMSE) on Training Data:", rmse)


Mean Absolute Error (MAE) on Training Data: 0.9035808295589178
Root Mean Squared Error (RMSE) on Training Data: 1.0756151073745435


### Step 6: Parameter Tuning
- Experiment with different parameters such as similarity threshold, neighborhood size, and similarity metric to optimize the performance of our ItemKNN algorithm.
- Use cross-validation or other techniques to tune these parameters and avoid overfitting.

1. **Import Necessary Libraries:**
   - We import the required libraries for performing grid search cross-validation (`GridSearchCV`), creating custom scorers (`make_scorer`), and utilizing the `NearestNeighbors` algorithm.

2. **Define Cosine Similarity Function:**
   - We define a custom function `cosine_similarity` to compute the cosine similarity between two vectors. This function calculates the dot product of the vectors and divides it by the product of their norms.

3. **Define Custom Scorer:**
   - We create a custom scorer `cosine_similarity_scorer` using `make_scorer`, which enables us to use cosine similarity as the scoring metric during grid search cross-validation.

4. **Define Parameter Grid:**
   - We specify a parameter grid `param_grid` containing the hyperparameters to be tuned. In this case, we're tuning the number of neighbors (`n_neighbors`) and the distance metric (`metric`) for the `NearestNeighbors` algorithm.

5. **Initialize NearestNeighbors Model:**
   - We initialize the `NearestNeighbors` model without specifying any hyperparameters.

6. **Create GridSearchCV Object:**
   - We create a `GridSearchCV` object named `grid_search` with the specified parameter grid, cross-validation strategy (5-fold cross-validation), and custom scoring metric (`cosine_similarity_scorer`).

7. **Fit the Data:**
   - We fit the `item_user_matrix` data to the `grid_search` object to perform hyperparameter tuning. `item_user_matrix` typically contains the item-item similarity matrix computed using collaborative filtering techniques.

8. **Get Best Hyperparameters:**
   - After fitting the data, we retrieve the best hyperparameters selected by the grid search using the `best_params_` attribute of the `grid_search` object.

9. **Print Best Parameters:**
   - Finally, we print the best hyperparameters obtained from the grid search.



`scikit-learn (sklearn)`:

- Scikit-learn is a popular machine learning library in Python that provides simple and efficient tools for data analysis and modeling.
- It includes various modules for tasks such as classification, regression, clustering, dimensionality reduction, and model selection.
- The GridSearchCV class from scikit-learn is used for hyperparameter tuning through grid search along with cross-validation.
- The make_scorer function allows us to create a custom scoring function for use with GridSearchCV.
- The NearestNeighbors class provides functionality for unsupervised nearest neighbors learning, which can be used for tasks such as finding k-nearest neighbors for a given data point.

In [60]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import make_scorer
import numpy as np

# Define a custom scorer based on cosine similarity defined above
cosine_similarity_scorer = make_scorer(cosine_similarity)

# Define the parameter grid
param_grid = {
    'n_neighbors': [5, 10, 15, 30, 40],
    'metric': ['cosine', 'euclidean']
}

# Initialize NearestNeighbors model
knn_model = NearestNeighbors()

# Create the GridSearchCV object
grid_search = GridSearchCV(knn_model, param_grid, cv=5, scoring=cosine_similarity_scorer)

# Fit the data to perform hyperparameter tuning
grid_search.fit(train_matrix)  # train_matrix contains item-user matrix for the train set

# Get the best hyperparameters
best_params = grid_search.best_params_

print("Best parameters:", best_params)


c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py", line 980, in _score
    scores = scorer(estimator, X_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py", line 980, in _score
    scores = scorer(estimator, X_test, **score_

GridSearchCV(cv=5, estimator=NearestNeighbors(),
             param_grid={'metric': ['cosine', 'euclidean'],
                         'n_neighbors': [5, 10, 15, 30, 40]},
             scoring=make_scorer(cosine_similarity, response_method='predict'))

Best parameters: {'metric': 'cosine', 'n_neighbors': 5}


This code sets up a GridSearchCV object to perform hyperparameter tuning using the train set (train_matrix). It explores different combinations of hyperparameters specified in the param_grid, evaluates them using 5-fold cross-validation (cv=5), and uses the cosine_similarity scorer to optimize the model's performance based on cosine similarity. Finally, it prints the best hyperparameters found during the search.

We have to perform the calculation of the predicted ratings for the train-val set to generate predicted ratings in order to evaluate the performance of our trained model on a validation dataset. 

If we computed the similarity matrix again specifically for the train_val set, it would essentially mean that we are using a different set of similarity measures for predicting ratings compared to what we used during training. This approach could lead to inconsistencies and potentially degrade the performance of our model. Here's what could happen:

1. **Inconsistency**: The similarity measures computed for the train_val set might differ from those computed for the training set due to variations in the data. As a result, the predicted ratings based on these new similarity measures may not align well with the predictions made during training, leading to inconsistency in the model's behavior.

2. **Overfitting**: Computing a new similarity matrix specifically for the train_val set might lead to overfitting on the validation data. The model may capture noise or idiosyncrasies present in the train_val set, which may not generalize well to unseen data.

3. **Increased Complexity**: Computing the similarity matrix again for the train_val set adds computational complexity and redundancy, especially if the similarity computation process is resource-intensive. This can result in longer training times and increased resource utilization.

Overall, it's generally recommended to use the same similarity measures or neighborhood definitions for both training and validation sets to ensure consistency and generalizability of the model.

In [61]:
# Initialize an empty DataFrame to store predicted ratings for train_val
predicted_ratings_val = pd.DataFrame(index=val_matrix.index, columns=val_matrix.columns)

# Iterate over each user and their ratings in the train_val set
for user_id, user_ratings in val_matrix.iterrows():
    for movie_id, rating in user_ratings.items():
        
        # Check if the movie has a neighborhood defined
        if movie_id in item_neighborhoods_train:
            neighborhood = item_neighborhoods_train[movie_id]
            
            # Filter out movies from the neighborhood that the user has rated in the train set
            filtered_neighborhood = [neighbor_movie_id for neighbor_movie_id in neighborhood if neighbor_movie_id in user_ratings.index and user_ratings.loc[neighbor_movie_id] != 0]
            
            # Check if there are valid indices in the filtered neighborhood
            if len(filtered_neighborhood) > 0:
                # Calculate the predicted rating for the target movie based on the neighborhood
                neighbor_ratings = [user_ratings.loc[neighbor_movie_id] for neighbor_movie_id in filtered_neighborhood]
                
                # Calculate the mean rating using only the movies in the neighborhood that have been rated by the user
                predicted_rating = np.mean(neighbor_ratings)
            else:
                # If the filtered neighborhood is empty, assign the mean rating of all movies rated by the user
                predicted_rating = user_ratings[user_ratings != 0].mean()
            
            # Assign the predicted rating to the corresponding cell in the DataFrame
            predicted_ratings_val.at[user_id, movie_id] = predicted_rating

# Fill NaN values with mean ratings across all users
predicted_ratings_val.fillna(predicted_ratings_val.mean().mean(), inplace=True)


C:\Users\Jaume\AppData\Local\Temp\ipykernel_12960\2894517846.py:33: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  predicted_ratings_val.fillna(predicted_ratings_val.mean().mean(), inplace=True)


In [62]:
predicted_ratings_val.head()

predicted_ratings_val.shape

,30,33,44,45,50,52,66,68,71,77,...,13576,13581,13594,13596,13625,13626,13629,13656,13668,13673
2073923,5.0,5.000000,5.000000,5.0,5.000000,5.000000,5.000000,3.557526,5.000000,5.000000,...,5.000000,5.0,5.000000,5.000000,5.0,5.000000,5.000000,5.000000,5.000000,5.0
940615,4.0,4.000000,4.000000,4.0,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,...,4.000000,4.0,4.000000,4.000000,4.0,4.000000,4.000000,4.000000,4.000000,4.0
564895,3.0,3.666667,3.666667,3.0,3.666667,3.666667,3.666667,3.666667,3.666667,3.666667,...,3.666667,4.0,3.666667,3.666667,3.0,3.666667,3.557526,3.666667,3.666667,4.0
332086,3.5,3.500000,3.500000,3.5,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000,...,3.500000,3.5,3.000000,3.500000,3.5,3.500000,3.500000,3.000000,3.500000,3.5
609601,4.0,4.000000,4.000000,4.0,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,...,4.000000,4.0,3.557526,4.000000,4.0,4.000000,4.000000,4.000000,4.000000,4.0


(2726, 139)

Now we evaluate the model with the Validation item-user matrix.

In [63]:
# Now, we can evaluate the model using the evaluate_model function
mae, rmse = evaluate_model(val_matrix.to_numpy(), predicted_ratings_val)

print("Mean Absolute Error (MAE) on Validation Data:", mae)
print("Root Mean Squared Error (RMSE) on Validation Data:", rmse)


Mean Absolute Error (MAE) on Validation Data: 0.9199600117832204
Root Mean Squared Error (RMSE) on Validation Data: 1.091588442541076


### Retraining of the model with the Train-Val set

In [64]:
# After the model evaluation on the validation set, we recalculate the similarity matrix
# of the model on the train-val data
item_similarity_matrix_train_val = calculate_item_similarity_matrix(train_val)

# We combine the similarity matrices (the retrained one and the genres)
item_similarity_matrix_train_val = combine_similarity_matrices(item_similarity_matrix_train_val, item_similarity_genres_matrix)

# We calculate the item neighborhoods for the train_val data 
item_neighborhoods_train_val = neighborhood_selection(item_user_array, index_to_movie_id, item_similarity_matrix_train_val)

### Step 7: Deployment

Once we have come up with the best parameters possible and trained the model with the whole train_validation set, we will test it on the test set.

In [65]:
test_matrix

,30,33,44,45,50,52,66,68,71,77,...,13576,13581,13594,13596,13625,13626,13629,13656,13668,13673
1280428,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
226770,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2134424,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
922551,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
140498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1648640,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
630594,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001195,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1355675,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [66]:
# Check if there are NaN values in the test matrix
test_matrix.isnull().sum().sum()

0

In [67]:
# Now, we can evaluate the model's predicted ratings on the test set

# Initialize an empty DataFrame to store predicted ratings
predicted_ratings_test = pd.DataFrame(index=test_matrix.index, columns=test_matrix.columns)

# Iterate over each user-item pair in the training set
for user_id, user_ratings in test_matrix.iterrows():
    for movie_id, rating in user_ratings.items():
        
        # Check if the movie has a neighborhood defined
        if movie_id in item_neighborhoods_train_val:
            neighborhood = item_neighborhoods_train_val[movie_id]
            
            # Filter out movies from the neighborhood that the user has rated in the training set
            filtered_neighborhood = [neighbor_movie_id for neighbor_movie_id in neighborhood if neighbor_movie_id in user_ratings.index and user_ratings.loc[neighbor_movie_id] != 0]
            
            # Check if there are valid indices in the filtered neighborhood
            if len(filtered_neighborhood) > 0:
                # Calculate the predicted rating for the target movie based on the neighborhood
                neighbor_ratings = [user_ratings.loc[neighbor_movie_id] for neighbor_movie_id in filtered_neighborhood]
                
                # Calculate the mean rating using only the movies in the neighborhood that have been rated by the user
                predicted_rating = np.mean(neighbor_ratings)
            else:
                # If the filtered neighborhood is empty, assign the mean rating of all movies rated by the user
                predicted_rating = user_ratings[user_ratings != 0].mean()
            
            # Assign the predicted rating to the corresponding cell in the DataFrame
            predicted_ratings_test.at[user_id, movie_id] = predicted_rating

# Fill NaN values with mean ratings across all users
predicted_ratings_test.fillna(predicted_ratings_test.mean().mean(), inplace=True)

# Now, we can evaluate the model's predicted ratings on the test set
mae, rmse = evaluate_model(test_matrix.to_numpy(), predicted_ratings_test.to_numpy())

print("Mean Absolute Error (MAE) on Test Data using Predicted Ratings from Test Set:", mae)
print("Root Mean Squared Error (RMSE) on Test Data using Predicted Ratings from Test Set:", rmse)


Mean Absolute Error (MAE) on Test Data using Predicted Ratings from Test Set: 0.8955634954880128
Root Mean Squared Error (RMSE) on Test Data using Predicted Ratings from Test Set: 1.0643290842670106


C:\Users\Jaume\AppData\Local\Temp\ipykernel_12960\2021144940.py:36: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  predicted_ratings_test.fillna(predicted_ratings_test.mean().mean(), inplace=True)


In [68]:
top_recommendations_test = []

# Iterate over each row (user) in the test_matrix
for user_id in test_matrix.index:
    # Get the actual user ID corresponding to the user index
    recommendations = get_top_recommendations(user_id, predicted_ratings_test, df)
    top_recommendations_test.append(recommendations)

# Output: List of top movie recommendations for each user in the test matrix
top_recommendations_test

# print size of top_recommendations
print("Size of top_recommendations_test:", len(top_recommendations_test))



[["Something's Gotta Give",
  'Striptease',
  'Final Justice',
  'Legend of Crystania: The Chaos Ring',
  'Full Metal Panic FUMOFFU'],
 ["Something's Gotta Give",
  'Benedict Arnold: A Question of Honor',
  'Striptease',
  'Kazaam',
  'Legend of Crystania: The Chaos Ring'],
 ["Something's Gotta Give",
  'Benedict Arnold: A Question of Honor',
  'Striptease',
  'Kazaam',
  'Legend of Crystania: The Chaos Ring'],
 ["Something's Gotta Give",
  'Striptease',
  'Final Justice',
  'Legend of Crystania: The Chaos Ring',
  'Full Metal Panic FUMOFFU'],
 ["Something's Gotta Give",
  'Striptease',
  'Final Justice',
  'Legend of Crystania: The Chaos Ring',
  'Full Metal Panic FUMOFFU'],
 ["Something's Gotta Give",
  'Benedict Arnold: A Question of Honor',
  'Striptease',
  'Kazaam',
  'Legend of Crystania: The Chaos Ring'],
 ["Something's Gotta Give",
  'Benedict Arnold: A Question of Honor',
  'Striptease',
  'Kazaam',
  'Legend of Crystania: The Chaos Ring'],
 ["Something's Gotta Give",
  'Stri

Size of top_recommendations_test: 3408


# Item-based Classification (ItemKNN)

### Step 1: Data preparation
- Ensure we have a dataset with user-item interactions. Each interaction should include the user ID, item ID, and the corresponding label or class for classification.
- Preprocess the data as needed, including handling missing values, encoding categorical variables, and splitting into training and testing sets.

For this step we are going to resuse the `item_user_matrix` that we created in the first model built. 

### Step 2: Compute Item Similarity


In [69]:
item_similarity_matrix_train.shape

(139, 139)

In [70]:
import numpy as np

def item_neighborhood_selection(similarity_matrix, k=None, threshold=None, item_ids=None):
    """
    Select a subset of similar items for each item based on similarity matrix.

    Parameters:
        similarity_matrix (numpy.ndarray): Item-item similarity matrix.
        k (int): Number of similar items to select (optional).
        threshold (float): Similarity threshold for selecting similar items (optional).
        item_ids (list): List of item IDs corresponding to rows/columns of the similarity matrix.

    Returns:
        dict: Dictionary containing similar items for each item.
    """
    num_items = similarity_matrix.shape[0]
    item_neighborhood = {}

    for i in range(num_items):
        if k is not None:
            # Select top-k similar items (excluding the item itself)
            similar_items_indices = np.argsort(similarity_matrix[i])[::-1][:k]
        elif threshold is not None:
            # Select items with similarity above threshold (excluding the item itself)
            similar_items_indices = np.where(similarity_matrix[i] > threshold)[0]

        # Remove the item itself from the neighborhood
        similar_items_indices = similar_items_indices[similar_items_indices != i]

        # Get the item ID corresponding to the current index
        current_item_id = item_ids[i] if item_ids is not None else i

        # Get the item IDs for the neighborhood
        neighborhood_item_ids = [item_ids[index] for index in similar_items_indices]

        # Store the similar items with the item's ID as the key
        item_neighborhood[current_item_id] = neighborhood_item_ids

    return item_neighborhood

# We have an item-item similarity matrix 'item_similarity_matrix_train'
# and we want to select top-5 similar items for each item
k = 5
item_neighborhoods_classification_train = item_neighborhood_selection(item_similarity_matrix_train, k=k, item_ids=index_to_movie_id)

# Check if there are any items with empty neighborhoods
empty_neighborhoods = []

for item_id, neighborhood in item_neighborhoods_classification_train.items():
    if not neighborhood:
        empty_neighborhoods.append(item_id)

if empty_neighborhoods:
    print("Items with empty neighborhoods:", empty_neighborhoods)
else:
    print("No items with empty neighborhoods found.")


No items with empty neighborhoods found.


In [71]:
len(item_neighborhoods_classification_train)

139

### Step 3: Ratings classification selection

In [72]:
import numpy as np
import pandas as pd

def recommend_movies(train_matrix, item_neighborhoods, movie_id_to_index, num_movies=5):
    """
    Recommend top movies for each user based on item neighborhood.

    Parameters:
        train_matrix (np.ndarray or pd.DataFrame): Matrix containing user-item ratings in the train set.
        item_neighborhoods (dict): Dictionary containing similar items for each item.
        movie_id_to_index (dict): Dictionary mapping movie IDs to their corresponding indices in the train_matrix.
        num_movies (int): Number of movies to recommend for each user.

    Returns:
        pd.DataFrame: DataFrame containing predicted ratings for recommended movies for each user.
    """
    # Convert train_matrix to DataFrame if it's a numpy array
    if isinstance(train_matrix, np.ndarray):
        train_matrix = pd.DataFrame(train_matrix, index=np.arange(train_matrix.shape[0]), columns=np.arange(train_matrix.shape[1]))

    # Initialize an empty DataFrame to store predicted ratings
    predicted_ratings = pd.DataFrame(index=train_matrix.index, columns=train_matrix.columns)

    # Iterate over each user-item pair in the training set
    for user_id, user_ratings in train_matrix.iterrows():
        for movie_id, rating in user_ratings.items():
            
            # Check if the movie has a neighborhood defined
            if movie_id in item_neighborhoods:
                neighborhood = item_neighborhoods[movie_id]
                
                # Initialize a flag to track if the rating has been predicted
                rating_predicted = False
                
                # Iterate over the items in the neighborhood
                for neighbor_movie_id in neighborhood:
                    # Check if the neighbor movie has been rated by the user
                    if neighbor_movie_id in user_ratings.index and user_ratings.loc[neighbor_movie_id] != 0:
                        # Use the rating of the closest item in the neighborhood
                        predicted_rating = user_ratings.loc[neighbor_movie_id]
                        rating_predicted = True
                        break
                
                # If none of the items in the neighborhood have been rated, use the user's average rating
                if not rating_predicted:
                    if user_ratings[user_ratings != 0].empty:
                        # If the user hasn't rated any movies, use the global mean rating
                        predicted_rating = train_matrix[train_matrix != 0].mean().mean()
                    else:
                        # Use the average of the ratings given by the user
                        predicted_rating = user_ratings[user_ratings != 0].mean()
                
                # Assign the predicted rating to the corresponding cell in the DataFrame
                predicted_ratings.at[user_id, movie_id] = predicted_rating

    # Fill NaN values with mean ratings across all users
    predicted_ratings.fillna(predicted_ratings.mean().mean(), inplace=True)

    # Set the index of the DataFrame as the user ID of the train set
    predicted_ratings.index = train_matrix.index

    # Display the head of the predicted_ratings DataFrame
    predicted_ratings.head()

    return predicted_ratings

# Recommend movies for each user in the train set
predicted_ratings_classification_train = recommend_movies(train_matrix, item_neighborhoods_classification_train, movie_id_to_index)

# Print the shape of the predicted ratings matrix
print("Shape of predicted ratings matrix:", predicted_ratings_classification_train.shape)


Shape of predicted ratings matrix: (10903, 139)


In [73]:
type(train_matrix)

pandas.core.frame.DataFrame

In [74]:
predicted_ratings_classification_train

,30,33,44,45,50,52,66,68,71,77,...,13576,13581,13594,13596,13625,13626,13629,13656,13668,13673
2227565,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
1950639,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
1093688,3.0,3.5,3.5,3.0,3.5,3.5,3.5,3.5,3.5,3.5,...,4.0,3.5,4.0,4.0,3.0,3.5,3.5,3.5,3.5,3.5
2285835,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2085211,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,...,4.0,3.5,4.0,4.0,3.5,3.5,3.5,3.5,3.5,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2627144,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
1599231,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
1849553,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
1298946,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


### Step 4: Recommendations Generation

In [75]:
import pandas as pd

# Convert predicted_ratings_classification_train to a pandas DataFrame if it's a numpy array
if isinstance(predicted_ratings_classification_train, np.ndarray):
    # Use the length of train_matrix as the number of rows and items as columns
    num_users, num_items = train_matrix.shape
    predicted_ratings_classification_train = pd.DataFrame(predicted_ratings_classification_train, index=range(num_users), columns=range(num_items))

# Initialize a dictionary to store top recommendations for each user
top_recommendations_per_user = {}

# Iterate over each user in the train matrix
for user_id in train_matrix.index:
    # Get the top recommendations for the current user using predicted_ratings_classification_train
    recommendations = get_top_recommendations(user_id, predicted_ratings_classification_train, df)
    
    # Store the recommendations in the dictionary
    top_recommendations_per_user[user_id] = recommendations

# Print the top 5 recommendations for each user
for user_id, recommendations in top_recommendations_per_user.items():
    print(f"User {user_id} recommendations:", recommendations)


User 2227565 recommendations: ["Something's Gotta Give", 'Striptease', 'Final Justice', 'Legend of Crystania: The Chaos Ring', 'Full Metal Panic FUMOFFU']
User 1950639 recommendations: ['Benedict Arnold: A Question of Honor', 'Striptease', 'Aqua Teen Hunger Force: Vol. 1', 'Kazaam', 'Legend of Crystania: The Chaos Ring']
User 1093688 recommendations: ['Monument Ave.', 'Red Lights', "I'll Sleep When I'm Dead", 'True Romance', 'Gang Related']
User 2285835 recommendations: ["Something's Gotta Give", 'Striptease', 'Final Justice', 'Legend of Crystania: The Chaos Ring', 'Full Metal Panic FUMOFFU']
User 2085211 recommendations: ['Monument Ave.', 'Cold Blooded', 'The Killing', 'Striptease', 'A Kiss Before Dying']
User 990639 recommendations: ['Benedict Arnold: A Question of Honor', 'Striptease', 'Aqua Teen Hunger Force: Vol. 1', 'Kazaam', 'Legend of Crystania: The Chaos Ring']
User 585137 recommendations: ['Black Moon Rising', 'Cold Mountain: Bonus Material', "I'll Sleep When I'm Dead", 'Vide

### Step 5: Model Evaluation (Item KNN Classification)

In [76]:
import numpy as np
import pandas as pd

# predicted_ratings_classification_train is our predicted ratings DataFrame or numpy array

# Ensure predicted_ratings_classification_train is a numpy array
if isinstance(predicted_ratings_classification_train, pd.DataFrame):
    predicted_ratings_classification_train = predicted_ratings_classification_train.to_numpy()

# train_matrix is the numpy array representing the item-user matrix

# Ensure train_matrix is a 2D array
if isinstance(train_matrix, pd.DataFrame):
    train_matrix = train_matrix.to_numpy()

if train_matrix.ndim == 1:
    train_matrix = np.expand_dims(train_matrix, axis=0)

# Evaluate the model
mae, rmse = evaluate_model(train, predicted_ratings_classification_train)

print("Mean Absolute Error (MAE) on Training Data:", mae)
print("Root Mean Squared Error (RMSE) on Training Data:", rmse)


Mean Absolute Error (MAE) on Training Data: 0.21168058871656315
Root Mean Squared Error (RMSE) on Training Data: 0.4839313793658147


### Step 6: Parameter Tuning

Experiment with different parameters such as similarity threshold, neighborhood size, and similarity metric to optimize the performance of our ItemKNN algorithm.
Use techniques like cross-validation to tune these parameters and avoid overfitting.

In [77]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import make_scorer
import numpy as np

# Define a custom scorer based on cosine similarity defined above
cosine_similarity_scorer = make_scorer(cosine_similarity)

# Define the parameter grid
param_grid = {
    'n_neighbors': [5, 10, 15, 30, 40],
    'metric': ['cosine', 'euclidean']
}

# Initialize NearestNeighbors model
knn_model = NearestNeighbors()

# Create the GridSearchCV object
grid_search = GridSearchCV(knn_model, param_grid, cv=5, scoring=cosine_similarity_scorer)

# Fit the data to perform hyperparameter tuning
grid_search.fit(val_matrix)  # val_matrix contains item-user matrix

# Get the best hyperparameters
best_params = grid_search.best_params_

print("Best parameters:", best_params)


c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py", line 980, in _score
    scores = scorer(estimator, X_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py", line 980, in _score
    scores = scorer(estimator, X_test, **score_

GridSearchCV(cv=5, estimator=NearestNeighbors(),
             param_grid={'metric': ['cosine', 'euclidean'],
                         'n_neighbors': [5, 10, 15, 30, 40]},
             scoring=make_scorer(cosine_similarity, response_method='predict'))

Best parameters: {'metric': 'cosine', 'n_neighbors': 5}


We are now going to obtain again the ratings for the full val set to evaluate the performance. 

In [78]:
# Function to recalculate predictions for a given matrix using the trained model
def recalculate_predictions(matrix, item_neighborhoods):
    """
    Recalculate predictions for a given matrix using the trained model.

    Parameters:
        matrix (pd.DataFrame): Matrix containing user-item ratings.

    Returns:
        pd.DataFrame: DataFrame containing recalculated predicted ratings for each user.
    """
    # Initialize an empty DataFrame to store recalculated predicted ratings
    recalculated_predictions = pd.DataFrame(index=matrix.index, columns=matrix.columns)

    # Iterate over each user-item pair in the matrix
    for user_id, user_ratings in matrix.iterrows():
        for movie_id, rating in user_ratings.items():
        
            # Check if the movie has a neighborhood defined
            if movie_id in item_neighborhoods:
                neighborhood = item_neighborhoods[movie_id]

                # Initialize a flag to track if the rating has been predicted
                rating_predicted = False

                # Iterate over the items in the neighborhood
                for neighbor_movie_id in neighborhood:
                    # Check if the neighbor movie has been rated by the user
                    if neighbor_movie_id in user_ratings.index and user_ratings.loc[neighbor_movie_id] != 0:
                        # Use the rating of the closest item in the neighborhood
                        predicted_rating = user_ratings.loc[neighbor_movie_id]
                        rating_predicted = True
                        break

                # If none of the items in the neighborhood have been rated, use the user's average rating
                if not rating_predicted:
                    if user_ratings[user_ratings != 0].empty:
                        # If the user hasn't rated any movies, use the global mean rating
                        predicted_rating = matrix[matrix != 0].mean().mean()
                    else:
                        # Use the average of the ratings given by the user
                        predicted_rating = user_ratings[user_ratings != 0].mean()

                # Assign the predicted rating to the corresponding cell in the DataFrame
                recalculated_predictions.at[user_id, movie_id] = predicted_rating

    # Fill NaN values with mean ratings across all users
    recalculated_predictions.fillna(recalculated_predictions.mean().mean(), inplace=True)

    return recalculated_predictions


# Recalculate predictions for the validation set (train_val_matrix)
recalculated_val_predictions = recalculate_predictions(val_matrix, item_neighborhoods_classification_train)


In [79]:
recalculated_val_predictions

,30,33,44,45,50,52,66,68,71,77,...,13576,13581,13594,13596,13625,13626,13629,13656,13668,13673
2073923,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
940615,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
564895,3.0,3.666667,3.666667,3.0,3.666667,3.666667,3.666667,3.666667,3.666667,3.666667,...,3.666667,3.666667,3.666667,3.666667,3.0,3.666667,3.666667,3.666667,3.666667,4.0
332086,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,...,3.5,3.5,3.0,3.5,3.5,3.5,3.5,3.0,3.5,3.5
609601,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2050389,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1121700,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
551401,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
1118120,5.0,4.5,4.5,5.0,4.5,4.5,4.5,4.5,4.5,4.5,...,4.5,4.5,4.5,4.5,5.0,4.5,4.5,4.5,4.5,4.5


In [80]:
# Evaluate the model
mae, rmse = evaluate_model(val, recalculated_val_predictions)

print("Mean Absolute Error (MAE) on Train-Validation Data:", mae)
print("Root Mean Squared Error (RMSE) on Train-Validation Data:", rmse)

Mean Absolute Error (MAE) on Train-Validation Data: 0.2054333851181229
Root Mean Squared Error (RMSE) on Train-Validation Data: 0.4768791699655354


### Step 7: Deployment

Once we have all the metrics, we have to recalculate the ratings for the test set, we are going to repeat the same process for the test set.

In [81]:
# We have to reobtain the neighborhood for the train_val data
# We will use the already calculated item-item similarity matrix for train_val data
item_neighborhoods_classification_train_val = item_neighborhood_selection(item_similarity_matrix_train_val, k=5, item_ids=index_to_movie_id)

# We combine the similarity matrices (the retrained one and the genres)
item_neighborhoods_classification_train_val = combine_similarity_matrices(item_neighborhoods_classification_train_val, item_similarity_genres_matrix)

# Recalculate predictions for the validation set (test matrix)
recalculated_test_predictions = recalculate_predictions(test_matrix, item_neighborhoods_classification_train_val)

KeyboardInterrupt: 

We also have to evaluate the metrics for the test set.

In [ ]:
# Evaluate the model
mae, rmse = evaluate_model(test, recalculated_test_predictions)

print("Mean Absolute Error (MAE) on Test Data:", mae)
print("Root Mean Squared Error (RMSE) on Train-Validation Data:", rmse)

Mean Absolute Error (MAE) on Test Data: 0.9021627996092461
Root Mean Squared Error (RMSE) on Train-Validation Data: 1.072702296516033
